In [1]:
from flask import Flask, render_template, request
from PIL import Image
from flask import send_from_directory
import re
import tensorflow as tf
import os
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt



In [2]:
import socket
import threading
import uuid
from typing import Any, Callable, cast, Optional
#from flask_cors import cross_origin
from werkzeug.serving import run_simple

In [3]:
checkpoint_path = 'checkpoints/weights.99-0.00.hdf5'

In [4]:
input_shape = (256,256,3)


In [5]:

def getModel():
    
    pretrained_model = Xception(input_shape = input_shape,
                weights = 'imagenet',
                layers=tf.keras.layers,
                include_top = False)
    pretrained_model.trainable = False
    
    for layer in pretrained_model.layers:
        layer.trainable = False
    
    model = Sequential()

    model.add(pretrained_model)

    model.add(Flatten())

    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(3))
    model.add(Activation('softmax'))

    model.summary()
    return model

In [6]:
model = getModel()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 8, 8, 2048)        20861480  
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_________________________________________________________________
dense (Dense)                (None, 512)               67109376  
_________________________________________________________________
activation (Activation)      (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 1539      
_________________________________________________________________
activation_1 (Activation)    (None, 3)                 0

In [7]:
model.load_weights(checkpoint_path)

In [8]:
# normalize images
train_datagen = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization= True)
    

target_size = (256,256)

In [9]:

classes = ['normal', 'COVID-19', 'other_pneumonia']
class_dict ={classes[i]: i for i in range(3)}

In [10]:
app = Flask(__name__, static_folder='./flask_app')

In [11]:
root_dir=os.path.join(os.getcwd(),'flask_app')

print(root_dir)


/lusnlsas/ramkik_data/covid19/covid/flask_app


In [12]:
@app.route('/assets/<path:filename>')
#@cross_origin()
def serve_static(filename):
    return send_from_directory(os.path.join(root_dir,'assets/'),filename)

@app.route('/result')
def serve_result():
    return send_from_directory(os.path.join(root_dir),'result.png')

@app.route('/')
def index_view():
    return render_template('index.html')

In [13]:
#@app.route('/predict',methods=['GET','POST'])
def predict():
    print(request.method)
    f = request.files['image']
    
    file_path = os.path.join(root_dir, 'uploads', f.filename)
    print(file_path)
    f.save(file_path)
    #file_path='/home/ramkik/covid19/static/uploads/E63574A7-4188-4C8D-8D17-9D67A18A1AFA.jpeg'
    image = Image.open(file_path).convert('RGB')

    image = transform(image)

    normalize_image = normalize(image)
    normalize_image = normalize_image.unsqueeze(0)
    if torch.cuda.is_available():
        normalize_image = normalize_image.cuda()
        
    #print(normalize_image.shape)
    output = model(normalize_image)
    print(output)

#     output = F.softmax(output)
#     print(output)
    prediction_score, pred_label_idx = torch.topk(output, 1)
    pred_label_idx.squeeze_()

    print(normalize_image.shape)
    mask, _ = gradcam(normalize_image)
    heatmap, result = visualize_cam(mask, image)
    mask_pp, _ = gradcam_pp(normalize_image)
    heatmap_pp, result_pp = visualize_cam(mask_pp, image, alpha=1.0)

    save_image(result, 'result.png', nrow=1)

#         grad = transforms.ToPILImage()(result_pp)
#         plt.imshow(grad)




    response = {}
    print('output : ',output)
    print(pred_label_idx)
    print(prediction_score)

    response['class'] = classes[pred_label_idx.item()]
    response['score'] = str(prediction_score.item())

    return response

In [14]:
thread = threading.Thread(target=lambda: run_simple('0.0.0.0', 8000, app))
thread.start()

 * Running on http://0.0.0.0:8000/ (Press CTRL+C to quit)


In [15]:
%tb

No traceback available to show.
10.208.0.229 - - [03/May/2020 20:00:20] "GET / HTTP/1.1" 200 -
10.208.0.229 - - [03/May/2020 20:00:21] "GET /assets/vendor/bootstrap/css/bootstrap.min.css HTTP/1.1" 200 -
10.208.0.229 - - [03/May/2020 20:00:29] "GET /assets/vendor/icofont/icofont.min.css HTTP/1.1" 200 -
10.208.0.229 - - [03/May/2020 20:00:41] "GET /assets/vendor/venobox/venobox.css HTTP/1.1" 200 -
10.208.0.229 - - [03/May/2020 20:00:41] "GET /assets/vendor/boxicons/css/boxicons.min.css HTTP/1.1" 200 -
10.208.0.229 - - [03/May/2020 20:00:41] "GET /assets/vendor/owl.carousel/assets/owl.carousel.min.css HTTP/1.1" 200 -
10.208.0.229 - - [03/May/2020 20:00:41] "GET /assets/vendor/aos/aos.css HTTP/1.1" 200 -
10.208.0.229 - - [03/May/2020 20:00:49] "GET /assets/css/style.css HTTP/1.1" 200 -
10.208.0.229 - - [03/May/2020 20:00:50] "GET /assets/vendor/jquery/jquery.min.js HTTP/1.1" 200 -
10.208.0.229 - - [03/May/2020 20:00:57] "GET /assets/img/hero-img.png HTTP/1.1" 200 -
10.208.0.229 - - [03/May